## Comma Separated List Output Parser
`Comma Separated List Output Parser`란?
- LLM의 출력을 쉼표로 구분된 리스트 형태로 파싱하는 도구입니다
- 문자열 응답을 Python 리스트로 자동 변환합니다
- 여러 항목을 나열하는 경우에 유용합니다 (예: 부대명 목록, 훈련 과목 목록 등)
- 간단한 텍스트 처리에 적합한 경량 파서입니다
- 예시: "보병, 포병, 기갑, 공병, 통신" → ["보병", "포병", "기갑", "공병", "통신"]


LLM 모델 객체 생성하기

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.1,  # 일관된 리스트 출력을 위해 낮은 temperature 설정
)

Comma Separated List Output Parser를 사용하지 않는 예시

In [ ]:
from langchain_core.prompts import PromptTemplate

# 기본 프롬프트 템플릿 정의
template = """
육군의 다양한 {category}를 나열해주세요.

각 항목을 쉼표로 구분하여 답변해주세요.
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["category"],
)

# 프롬프트 미리보기
sample_category = "병과"
formatted_prompt = prompt.format(category=sample_category)
print(formatted_prompt)


In [ ]:
chain = prompt | llm

result = chain.invoke({"category": "병과"})

print(result.content)
print(f"응답 타입: {type(result.content)}")

이러한 경우처럼 여러 항목을 나열하는 응답이 필요할 때,  
Comma Separated List Output Parser를 사용하면 문자열을 자동으로 리스트로 변환할 수 있다.

1. Comma Separated List Output Parser 생성하기

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Comma Separated List Output Parser 생성
parser = CommaSeparatedListOutputParser()

# 파서가 어떤 형식을 요구하는지 확인
print(parser.get_format_instructions())

### `get_format_instructions()` 메서드
- `CommaSeparatedListOutputParser`의 출력 형식을 LLM이 이해할 수 있는 명확한 지침으로 변환
- LLM이 쉼표로 구분된 리스트 형태로 응답하도록 안내

프롬프트 재정의
- `category` : 나열하고 싶은 카테고리 (사용자 입력을 받을 위치)
- `format_instructions` : parser로 형식 지정

In [ ]:
# 프롬프트 템플릿 재정의 (format_instructions 포함)
template = """
육군의 다양한 {category}를 나열해주세요.

{format_instructions}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["category"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 프롬프트 미리보기
sample_category = "병과"
formatted_prompt = prompt.format(category=sample_category)
print(formatted_prompt)


LCEL 체인 만들기
- 체인 구조: PromptTemplate → ChatOllama → CommaSeparatedListOutputParser

In [ ]:
# LCEL을 사용한 체인 생성: 프롬프트 → LLM → 파서
chain = prompt | llm | parser

# 테스트 데이터
test_category = "육군의 병과"

# 체인 실행
result = chain.invoke({"category": test_category})

print(f"파싱 결과: {result}")